In [1]:
import chromadb
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import datetime as dt

API_KEY = os.getenv('OAI_API_KEY')


In [2]:
df = pd.read_csv('./data/saigai_data.csv')
df = df.astype({'年': 'Int64', '月': 'Int64', '業種（中分類）': 'Int64', '業種（小分類）': 'Int64', '事故の型': 'Int64', '起因物（大分類）.2': 'Int64', '起因物（中分類）.2': 'Int64', '起因物（小分類）.2': 'Int64'})
df_new = df[['年号','年','月','災害状況','業種（大分類）.1']]

/var/folders/5s/lns070lx7x3_4_7qbhgzsj0w0000gn/T/ipykernel_13683/2698770361.py:1: DtypeWarning: Columns (7,14,16,17,18,22,24,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/saigai_data.csv')


In [3]:
client = chromadb.Client()

In [4]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=API_KEY,
                model_name="text-embedding-ada-002"
            )
collection = client.create_collection(name="disaster_collection",embedding_function=openai_ef)


In [7]:
df_dic = df_new[:20000].to_dict()
docs = list(df_dic['災害状況'].values())
# df_dicから'災害状況'キーを除いた辞書を取り出すコード
original_data = {key: value for key, value in df_dic.items() if key != '災害状況'}
metadata = [
    {key: value[index] for key, value in original_data.items()}
    for index in range(len(next(iter(original_data.values()))))
]

ids = ["id{}".format(i) for i in df_dic['災害状況'].keys()]



In [8]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids
)

BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [12]:
results = collection.query(
    query_texts=["データ"],
    n_results=10
)
results

{'ids': [['id89', 'id285', 'id164']],
 'distances': [[0.6319524049758911, 0.6319524049758911, 0.6319524049758911]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': 'その他の事業'}]],
 'embeddings': None,
 'documents': [['記載なし', '記載なし', '記載なし']],
 'uris': None,
 'data': None}